In [6]:
from confluent_kafka import SerializingProducer
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.avro import AvroSerializer
from confluent_kafka.serialization import IntegerSerializer
from confluent_kafka.serialization import StringSerializer
import configparser as cf
from datetime import datetime
from logistics_data_generator import generate_logistics_data

In [7]:

# Define Kafka configuration
kafka_config = {
    'bootstrap.servers': 'pkc-l7pr2.ap-south-1.aws.confluent.cloud:9092',
    'sasl.mechanisms': 'PLAIN',
    'security.protocol': 'SASL_SSL',
    'sasl.username': '2TC7MB4C64INXCNM',
    'sasl.password': 'bzj7MZ1TOn5rBt4Q2y+yxoitC52IXpHvhymia5Y1i3eCPH0gQzsYnNvwgcHAhRTq'
}


In [8]:
# Create a Schema Registry client
schema_registry_client = SchemaRegistryClient({
  'url': 'https://psrc-e8vk0.southeastasia.azure.confluent.cloud',
  'basic.auth.user.info': '{}:{}'.format('47JJSLPLZYEJU7JL', 'LSu4H9SFe8mgNnexAoQChOpWFqDQGQFnAO4KxW4fhexLpIYv2wgwsWVN/jIEVKSK')
})

In [9]:
# Fetch the latest Avro schema for the value
subject_name = 'fedex_logistics-value'
schema_str = schema_registry_client.get_latest_version(subject_name).schema.schema_str

In [10]:
# key_serializer = AvroSerializer(schema_registry_client=schema_registry_client, schema_str='{"type": "int"}')
# Create Avro Serializer for the value
key_serializer = StringSerializer()
avro_serializer = AvroSerializer(schema_registry_client, schema_str)

In [11]:
# Define the SerializingProducer
producer = SerializingProducer({
    'bootstrap.servers': kafka_config['bootstrap.servers'],
    'security.protocol': kafka_config['security.protocol'],
    'sasl.mechanisms': kafka_config['sasl.mechanisms'],
    'sasl.username': kafka_config['sasl.username'],
    'sasl.password': kafka_config['sasl.password'],
    'key.serializer': key_serializer,  # Key will be serialized as an integer
    'value.serializer': avro_serializer  # Value will be serialized as Avro
})

In [13]:
def delivery_report(err, msg):
    """
    Reports the failure or success of a message delivery.

    Args:
        err (KafkaError): The error that occurred on None on success.
        msg (Message): The message that was produced or failed.

    Note:
        In the delivery report callback the Message.key() and Message.value()
        will be the binary format as encoded by any configured Serializers and
        not the same object that was passed to produce().
        If you wish to pass the original object(s) for key and value to delivery
        report callback we recommend a bound callback or lambda where you pass
        the objects along.
    """
    if err is not None:
        print("Delivery failed for record {}: {}".format(msg.key(), err))
        return
    print('Record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [ ]:
def produce_messages_to_kafka():
    # dict_value['timestamp'] = datetime.strftime(dict_value['timestamp'], "%Y-%m-%dT%H:%M:%SZ")
    for _ in range(1750):
        dict_value = generate_logistics_data()
        producer.produce(topic="fedex_logistics",
                         key=str(dict_value['shipment_id']),
                         value=dict_value,
                         on_delivery=delivery_report)
        producer.flush()
    print("Messages are produces successfully")


if __name__ == "__main__":
    produce_messages_to_kafka()